### Merging the feature spaces of Sentinel 2 and VHR (TRIPLESAT) - Tiny Demo

#### Load the libraries we'll need
* **`datetime`** to transform column names that include dates
* **`pandas`** for dataframe handling
* **`numpy`** for fast numeric operations
* **`os`** for various interactions with the system, like safely joining paths

In [241]:
from datetime import datetime
import pandas as pd
import numpy as np
import os

#### Load the 2 tiny csv files
We are initially using tiny versions of the feature spaces to make experimentations faster and easier to interpret. They are called tiny because I've only kept the header and 4 rows selected in such way that the following transformations apply.

In [242]:
s2 = pd.read_csv('s2_tiny.csv', index_col='id')
triplesat = pd.read_csv('triplesat_tiny.csv', index_col='id')

The Sentinel-2 feature space, comes with a column that serves no purpose. It's called "Unnamed: 0" and it is probably a problem to be fixed on the corresponding Sentinel-2 scripts. But we don't need it, and thus, if it exists we can drop it.

In [243]:
# There is a weird 'Unnamed: 0' column in the S2 feature space.
# Check if it does exist, and drop it in that case.
if s2.columns[0] == 'Unnamed: 0':
    s2.drop(s2.columns[[0]], axis=1, inplace=True)
    print('Column "Unnamed: 0" was found in the S2 feature space and was dropped')
else:
    print('No unnecessary columns found in the S2 feature space')


Column "Unnamed: 0" was found in the S2 feature space and was dropped


#### Preview the 2 dataframes
Here is what the Sentinel-2 feature space looks like:

In [244]:
s2

,B02_83,B03_83,B04_83,B05_83,B06_83,B07_83,B08_83,B8A_83,B11_83,B12_83,...,B05_323,B06_323,B07_323,B08_323,B8A_323,B11_323,B12_323,NDVI_323,NDWI_323,PSRI_323
id,,,,,,,,,,,,,,,,,,,,,
1,641.72500,864.30000,845.65000,1357.5000,2244.7500,2450.5000,2696.8500,2781.9250,2414.0250,1630.35000,...,692.0294,1330.0588,1542.3235,1584.0000,1773.4117,986.9706,497.70587,721.106362,232.219458,63.752166
2,533.67303,807.45890,594.94867,1276.6349,3062.9340,3494.6025,3803.0308,3658.7068,1890.4912,1142.17300,...,1327.0361,2876.8618,3196.4983,3424.3948,3436.9473,1614.1431,916.92430,711.118775,359.281152,13.986358
9,493.35098,730.10583,488.44620,1149.4127,2855.7883,3286.1482,3564.6420,3455.8042,1752.2946,943.77780,...,1180.3225,3310.0972,3798.6843,4084.5393,4103.1160,1721.1212,877.28156,836.821455,407.088582,0.408822
10,436.41833,696.79370,431.42120,1166.3782,2967.1145,3412.8367,3662.0230,3612.5044,1860.3926,951.24927,...,1187.5472,3278.2349,3793.4270,3967.5930,4096.9570,1772.5187,907.36390,828.684542,382.409684,2.505891
114,431.49600,671.66930,381.00266,1058.1627,3252.2827,3945.4026,4224.0293,4221.4720,1969.8480,963.95200,...,1022.8000,3456.4294,4139.4640,4360.7200,4439.6533,1651.4694,793.55730,889.965538,450.626289,-1.115608
115,427.12122,665.34436,374.91183,1038.9724,3196.0828,3882.8430,4205.0576,4145.3115,1932.2837,951.36365,...,1002.4270,3274.5178,3926.6860,4134.6997,4189.2670,1560.3279,756.06335,882.495777,452.038512,-3.138859


And the triplesat feature space:

In [245]:
triplesat

,20170923094819_BLUE,20170923094819_GREEN,20170923094819_RED,20170923094819_NIR,20170923094822_BLUE,20170923094822_GREEN,20170923094822_RED,20170923094822_NIR
id,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,680.00000,442.5000,281.500,775.5000,NaN,NaN,NaN,NaN
10,161.50000,114.5000,71.500,180.0000,NaN,NaN,NaN,NaN
114,684.11730,560.9813,343.232,1246.4426,681.9089,558.08310,339.71313,1223.90080
115,713.56665,584.2889,372.550,1219.1888,528.8184,429.26816,271.10614,943.94415


#### Keep the common bands
As we can see above, we have many more columns in the Sentinel-2 feature space (572) than in the Triplesat feature space (8). This is expected and is caused by 2 reasons:
* We have 13 features for each Sentinel-1 acquisition, while 4 for each Triplesat acquisition
* The Sentinel-2 feature space comes from many acquisitions instead of just 2 which is the number of Triplesat acquisitions (used in this analysis)

There are 4 common bands between the 2 satellite sensors that correspond to the Blue, Green, Red and Near-Infrared wavelengths. According to these pages [for Sentinel-2](https://hatarilabs.com/ih-en/how-many-spectral-bands-have-the-sentinel-2-images) and [for Triplesat](https://www.satimagingcorp.com/satellite-sensors/triplesat-satellite/), the mapping is as follows:

| S2 Band | 3SAT Band | Name |
|---|:---:|:---|
|B02|1|Blue|
|B03|2|Green|
|B04|3|Red|
|B08|4|NIR|

We can initially see that the S2 columns are named after the convention `BAND_DOY`, where:
* `BAND` is the Sentinel-2 band, ie. `B02, B03, B04, ...`
* `DOY` is the day of year

In order to make the 2 dataframes compatible, we have to transform the column names so that they are using the same convention. It makes sense to do it in the following way:
* Keep only the S2 columns that have a corresponding 3SAT value, ie. `B02, B03, B04, B08`
* Convert the S2 names to actual band names (Blue, Green, Red, NIR)
* Convert the 3SAT dates to the day of year format

#### Keep the 4 Sentinel-2 bands
Let's create a list of the columns we want to keep.

In [246]:
# A column list with the S2 bands that we can keep
s2_columns = [col for col in s2.columns if
                col.startswith('B02') or
                col.startswith('B03') or
                col.startswith('B04') or
                col.startswith('B08')]

# Now only keep these columns
s2 = s2[s2_columns]
print('Columns that are incompatible for 1st-level fusion have been dropped from the S2 feature space')

Columns that are incompatible for 1st-level fusion have been dropped from the S2 feature space


And after keeping only them, here is how the Sentinel-2 feature space looks like.

In [247]:
s2

,B02_83,B03_83,B04_83,B08_83,B02_86,B03_86,B04_86,B08_86,B02_116,B03_116,...,B04_313,B08_313,B02_321,B03_321,B04_321,B08_321,B02_323,B03_323,B04_323,B08_323
id,,,,,,,,,,,,,,,,,,,,,
1,641.72500,864.30000,845.65000,2696.8500,484.82500,742.30000,761.97500,2546.2500,1832.888900,2062.389000,...,289.315296,1882.280000,171.882350,290.794130,256.676480,1584.000000,171.88235,290.79413,256.67648,1584.0000
2,533.67303,807.45890,594.94867,3803.0308,375.64460,675.77264,494.36835,3726.8777,520.222200,784.628970,...,337.406560,2651.885600,241.500750,490.362550,273.020800,2490.596000,537.88983,850.35364,578.12665,3424.3948
9,493.35098,730.10583,488.44620,3564.6420,284.04095,587.36176,330.35324,4151.3480,296.216300,611.542240,...,347.339597,4668.259383,363.167245,696.928583,359.755983,4201.283317,361.50684,692.48120,362.86008,4084.5393
10,436.41833,696.79370,431.42120,3662.0230,306.54440,613.44130,370.11176,3974.0230,279.301580,537.433375,...,352.459903,4570.754417,365.103637,700.349067,367.846717,4088.225283,363.47852,697.56730,371.69342,3967.5930
114,431.49600,671.66930,381.00266,4224.0293,282.40268,539.32800,277.84268,4347.9040,362.838672,669.807998,...,402.208000,4853.589400,213.373340,453.965330,201.472000,3720.453400,257.73868,569.61600,253.88266,4360.7200
115,427.12122,665.34436,374.91183,4205.0576,279.27548,523.16530,272.29752,4284.7656,324.868445,628.654975,...,417.366400,4637.289000,236.159770,510.013760,227.407710,4135.911600,268.36365,560.20660,258.08540,4134.6997


#### Rename S2 band columns
To keep a common naming convention between S2 and TripleSat, it makes sense to convert the first part of the S2 band names to the Name, as seen on the table shared above. Thus:
* `B02` → `BLUE`
* `B03` → `GREEN`
* `B04` → `RED`
* `B08` → `NIR`

In [248]:
# Replace each of the patterns
s2.columns = s2.columns.str.replace('B02','BLUE')
s2.columns = s2.columns.str.replace('B03','GREEN')
s2.columns = s2.columns.str.replace('B04','RED')
s2.columns = s2.columns.str.replace('B08','NIR')

This is how the S2 dataframe looks like after renaming

In [249]:
s2

,BLUE_83,GREEN_83,RED_83,NIR_83,BLUE_86,GREEN_86,RED_86,NIR_86,BLUE_116,GREEN_116,...,RED_313,NIR_313,BLUE_321,GREEN_321,RED_321,NIR_321,BLUE_323,GREEN_323,RED_323,NIR_323
id,,,,,,,,,,,,,,,,,,,,,
1,641.72500,864.30000,845.65000,2696.8500,484.82500,742.30000,761.97500,2546.2500,1832.888900,2062.389000,...,289.315296,1882.280000,171.882350,290.794130,256.676480,1584.000000,171.88235,290.79413,256.67648,1584.0000
2,533.67303,807.45890,594.94867,3803.0308,375.64460,675.77264,494.36835,3726.8777,520.222200,784.628970,...,337.406560,2651.885600,241.500750,490.362550,273.020800,2490.596000,537.88983,850.35364,578.12665,3424.3948
9,493.35098,730.10583,488.44620,3564.6420,284.04095,587.36176,330.35324,4151.3480,296.216300,611.542240,...,347.339597,4668.259383,363.167245,696.928583,359.755983,4201.283317,361.50684,692.48120,362.86008,4084.5393
10,436.41833,696.79370,431.42120,3662.0230,306.54440,613.44130,370.11176,3974.0230,279.301580,537.433375,...,352.459903,4570.754417,365.103637,700.349067,367.846717,4088.225283,363.47852,697.56730,371.69342,3967.5930
114,431.49600,671.66930,381.00266,4224.0293,282.40268,539.32800,277.84268,4347.9040,362.838672,669.807998,...,402.208000,4853.589400,213.373340,453.965330,201.472000,3720.453400,257.73868,569.61600,253.88266,4360.7200
115,427.12122,665.34436,374.91183,4205.0576,279.27548,523.16530,272.29752,4284.7656,324.868445,628.654975,...,417.366400,4637.289000,236.159770,510.013760,227.407710,4135.911600,268.36365,560.20660,258.08540,4134.6997


#### Rename band columns to a common convention
It will be good to convert the columns of the feature spaces following a common convention.
* The Sentinel-2 acquisitions, as already mentioned, follow the `BAND_DOY` convention
* The Triplesat acquisitions use names of the form `YYYYMMDDHHMMSS_BAND`

We will use the **`DOY_BAND`** convention so that later on, the columns will be sorted alphanumerically into an order that makes temporal sense. The `DOY` (Day Of Year) part will use preceding zeroes where needed (001-365/6), which means that the actual order will be "enforced" regardless of the number of non-zero digits.

#### Datetime conversion utility function
For the purpose of converting the date part of the columns into the output format we wish, let's create a utility function

In [250]:
# Utility function to convert date strings between datetime formats
# Arguments:
#   1. in_str
#   2. in_format
#   3. out_format
# Returns:
#      The date converted to the out_format (as a string)
#
def convert_date_string(in_str, in_format, out_format):
    # Initially convert input string to datetime object
    in_str = datetime.strptime(in_str, in_format)
    
    # Then, return it as a string again, after converting it to the out_format
    # This could also be a one-liner but it wouldn't be easily readable
    return in_str.strftime(out_format)

#### Rename TripleSAT columns to the common convention
The TripleSAT format of the date part of the column names is `YYYYMMDDhhmmss` which in python code translates to `%Y%m%d%H%M%S`

In [251]:
triplesat_date_in_format = '%Y%m%d%H%M%S'
triplesat_date_out_format = '%j'
new_colnames = []
for col in triplesat.columns:
    # separate the date and band parts of the triplesat column names
    date_part = col.split('_')[0]
    band_part = col.split('_')[1]

    new_date = convert_date_string(date_part, triplesat_date_in_format, triplesat_date_out_format)
    # convert to the output format by calling the utility function
    new_colnames.append('{0}_{1}'.format(new_date,band_part))

# Now assign the new names to the 3SAT columns
triplesat.columns = new_colnames

#### Rename Sentinel-2 columns to the common convention
We also want to make sure that instead of `BAND_DOY` we convert the sentinel-2 columns to `DOY_BAND`.

In [272]:
s2_date_in_format = '%j'
s2_date_out_format = '%j'
new_colnames = []
for col in s2.columns:
    # separate the date and band parts of the sentinel 2 column names
    date_part = col.split('_')[1]
    band_part = col.split('_')[0]

    new_date = convert_date_string(date_part, s2_date_in_format, s2_date_out_format)
    # convert to the output format by calling the utility function
    new_colnames.append('{0}_{1}'.format(new_date,band_part))

# Now assign the new names to the S2 columns
s2.columns = new_colnames

ValueError: time data 'BLUE' does not match format '%j'

Here is how the feature spaces now look like. This is Sentinel-2 after conversion:

In [253]:
s2

,083_BLUE,083_GREEN,083_RED,083_NIR,086_BLUE,086_GREEN,086_RED,086_NIR,116_BLUE,116_GREEN,...,313_RED,313_NIR,321_BLUE,321_GREEN,321_RED,321_NIR,323_BLUE,323_GREEN,323_RED,323_NIR
id,,,,,,,,,,,,,,,,,,,,,
1,641.72500,864.30000,845.65000,2696.8500,484.82500,742.30000,761.97500,2546.2500,1832.888900,2062.389000,...,289.315296,1882.280000,171.882350,290.794130,256.676480,1584.000000,171.88235,290.79413,256.67648,1584.0000
2,533.67303,807.45890,594.94867,3803.0308,375.64460,675.77264,494.36835,3726.8777,520.222200,784.628970,...,337.406560,2651.885600,241.500750,490.362550,273.020800,2490.596000,537.88983,850.35364,578.12665,3424.3948
9,493.35098,730.10583,488.44620,3564.6420,284.04095,587.36176,330.35324,4151.3480,296.216300,611.542240,...,347.339597,4668.259383,363.167245,696.928583,359.755983,4201.283317,361.50684,692.48120,362.86008,4084.5393
10,436.41833,696.79370,431.42120,3662.0230,306.54440,613.44130,370.11176,3974.0230,279.301580,537.433375,...,352.459903,4570.754417,365.103637,700.349067,367.846717,4088.225283,363.47852,697.56730,371.69342,3967.5930
114,431.49600,671.66930,381.00266,4224.0293,282.40268,539.32800,277.84268,4347.9040,362.838672,669.807998,...,402.208000,4853.589400,213.373340,453.965330,201.472000,3720.453400,257.73868,569.61600,253.88266,4360.7200
115,427.12122,665.34436,374.91183,4205.0576,279.27548,523.16530,272.29752,4284.7656,324.868445,628.654975,...,417.366400,4637.289000,236.159770,510.013760,227.407710,4135.911600,268.36365,560.20660,258.08540,4134.6997


And this is Triplesat after conversion:

In [254]:
triplesat

,266_BLUE,266_GREEN,266_RED,266_NIR,266_BLUE,266_GREEN,266_RED,266_NIR
id,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,680.00000,442.5000,281.500,775.5000,NaN,NaN,NaN,NaN
10,161.50000,114.5000,71.500,180.0000,NaN,NaN,NaN,NaN
114,684.11730,560.9813,343.232,1246.4426,681.9089,558.08310,339.71313,1223.90080
115,713.56665,584.2889,372.550,1219.1888,528.8184,429.26816,271.10614,943.94415


#### Multiple acquisitions per day of year
For the particular VHR acquisitions we are using, we notice that the column names are repetitive as they are of the same day of year. This is normal and it is a result of:
* the acquisitions being indeed of the same day (covering different areas with a small time difference of a few seconds)
* there is a small area containing parcels that is captured by both acquisitions (overlapping part of acquisitions)

We want to have only 1 value per day, and maybe the most obvious way is to abide by the following rules:
1. If all acquisitions (2 in this demo) are `NaN`, then the result is `NaN`
2. If `0<n<all` acquistions have values, then keep the average (`sum/n`), where `sum` is the sum of the non-`NaN` values.

There is a caveat for keeping the average of multiple acquisitions though. Some parcels in the edges of acquisitions have very small values. This is because of the acquisition-edges being black, which means that some or all of the parcel pixels are also black. So there are cases were a parcel is covered by 1 acquisition properly, but is on the very edge of the other. Thus, we may have a zonal statistic value of NIR 700 for example because of acquisition 1, and a NIR value of 20 because of acquisition 2 having many black pixels.

This is something that we need to take into account. There are 2 ways to handle it that initially come to mind:
1. Find the acquisition with the **highest** value and compare that value with the rest. If the other values have a difference of more than 5% (so they are smaller by more than 5%), then throw them away and calculate the mean with the rest.
2. Find the acquisition with the **highest** value and **_keep that one only_**.

In fact, later on, it might be a good idea to merge values further so that we have groups of wider timeframes. For example, instead of having 1 value per day, we could (and probably should) group values further and have 1 for every 5 days, or for a week or even more.

As we have to decide the way to move forward, we consider for the moment that the best option for these cases of overlaps is option 2. Thus, keep the highest value. This looks like the safest option that makes sure there is no unwanted skew coming from the average calculation.

#### Step 1: Prepare metadata to identify single and duplicate colnames
We will keep some data for each of the columns. That will be the name, and then, for each name, we will also keep the column indices of appearance. Duplicates will, thus, be the ones with more than 1 indices.

In [255]:
def prepareFusionMetadata(dataset):
    # We want each name once, so let's make it a set
    duplicate_colnames = set()
    # Get the dataset column names
    col_list = list(dataset.columns)

    # current is a helper variable for tranversing the columns
    # col_indexes is a dictionary which, for every column name with duplicates
    current = 0
    col_indexes = {}
    # Iterate on the dataset column names to find duplicates
    for col in col_list:
        # Add the column name in the dictionary if it doesn't already exist
        # together with the index of appearance
        if col not in col_indexes.keys():
            col_indexes[col] = [current]
        # otherwise, if it's already there, only add the current index of appearance
        else:
            col_indexes[col].append(current)

        # If a column that has not already been checked, find out if it appears more than once
        if col not in duplicate_colnames and col_list.count(col) > 1:
            # add it to the set of duplicates
            duplicate_colnames.add(col)

        current += 1

    dup_length = len(duplicate_colnames)
    if dup_length == 0:
        print('No duplicate columns.')
    elif dup_length%4 == 0:
        print('Length of the duplicate column names ({0}) is divisible by 4, which looks good.'.format(dup_length))
    else:
        print('Length of the duplicate column names ({0}) is not divisible by 4. Worth investigating this case')

    return col_indexes

colIndexes = prepareFusionMetadata(triplesat)

Length of the duplicate column names (4) is divisible by 4, which looks good.


Let's see what are the column names together with their appearance indexes:

In [256]:
print("Column\t\tIndexes")
for item in colIndexes:
    print('{0}:\t {1}'.format(item,colIndexes[item]))

Column		Indexes
266_BLUE:	 [0, 4]
266_GREEN:	 [1, 5]
266_RED:	 [2, 6]
266_NIR:	 [3, 7]


#### Step 2: Transverse through the columns and keep the appropriate values

We will now create a new dataframe:
1. As a starting point, the index is copied from the original to the new dataframe
2. For each column name, we are checking the indices that they appear in
  * unique columns appear once, so they have only 1 index, ie. unique acquisition for a single day
  * more than one indices indicate cases where we have overlapping areas and, thus, multiple acquisitions per day
3. We append the maximum value between the available values or NaN if all acquisitions are NaN

In [257]:
# Create the new dataframe and start by copying the index column from the original
df3s = pd.DataFrame(index=triplesat.index)

for item in colIndexes:
    # Get the corresponding columns only to a new dataframe
    print("Working on column: {0}".format(item))
    if len(colIndexes[item]) > 1:
        print(' ... more than 1 columns with this name, keeping the max values')
    temp_df = triplesat.iloc[:,colIndexes[item]]

    # Now insert the max values for each row of these columns into the new df
    df3s.insert(
        loc=len(df3s.columns),
        column=item,
        value=list(triplesat.iloc[:,colIndexes[item]].max(axis=1).values)
    )

Working on column: 266_BLUE
 ... more than 1 columns with this name, keeping the max values
Working on column: 266_GREEN
 ... more than 1 columns with this name, keeping the max values
Working on column: 266_RED
 ... more than 1 columns with this name, keeping the max values
Working on column: 266_NIR
 ... more than 1 columns with this name, keeping the max values


By now, we should have managed to bring the VHR dataset into a fuse-ready state. Here is it is:

In [258]:
df3s

,266_BLUE,266_GREEN,266_RED,266_NIR
id,,,,
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
9,680.00000,442.5000,281.500,775.5000
10,161.50000,114.5000,71.500,180.0000
114,684.11730,560.9813,343.232,1246.4426
115,713.56665,584.2889,372.550,1219.1888


### Merging VHR dataset with S2 dataset
So now that we have both the **S2** and the **3SAT** VHR datasets into a common format, we can merge them together.

In [259]:
triplesat_colIndexes = prepareFusionMetadata(df3s)
s2_colIndexes = prepareFusionMetadata(s2)

No duplicate columns.
No duplicate columns.


In [260]:
# check if any column appears in both dataframes
# thus, if we have same-day acquisition for both
# S2 and triplesat
if any([x in list(s2_colIndexes.keys()) for x in list(triplesat_colIndexes.keys())]):
    print('There is at least one day for which we have acquisitions from both satellites')
else:
    print('There is no common acquisition from the 2 satellites in these datasets')

There is at least one day for which we have acquisitions from both satellites


In [261]:
s2_colIndexes

{'083_BLUE': [0],
 '083_GREEN': [1],
 '083_RED': [2],
 '083_NIR': [3],
 '086_BLUE': [4],
 '086_GREEN': [5],
 '086_RED': [6],
 '086_NIR': [7],
 '116_BLUE': [8],
 '116_GREEN': [9],
 '116_RED': [10],
 '116_NIR': [11],
 '126_BLUE': [12],
 '126_GREEN': [13],
 '126_RED': [14],
 '126_NIR': [15],
 '146_BLUE': [16],
 '146_GREEN': [17],
 '146_RED': [18],
 '146_NIR': [19],
 '153_BLUE': [20],
 '153_GREEN': [21],
 '153_RED': [22],
 '153_NIR': [23],
 '156_BLUE': [24],
 '156_GREEN': [25],
 '156_RED': [26],
 '156_NIR': [27],
 '188_BLUE': [28],
 '188_GREEN': [29],
 '188_RED': [30],
 '188_NIR': [31],
 '191_BLUE': [32],
 '191_GREEN': [33],
 '191_RED': [34],
 '191_NIR': [35],
 '198_BLUE': [36],
 '198_GREEN': [37],
 '198_RED': [38],
 '198_NIR': [39],
 '201_BLUE': [40],
 '201_GREEN': [41],
 '201_RED': [42],
 '201_NIR': [43],
 '203_BLUE': [44],
 '203_GREEN': [45],
 '203_RED': [46],
 '203_NIR': [47],
 '216_BLUE': [48],
 '216_GREEN': [49],
 '216_RED': [50],
 '216_NIR': [51],
 '218_BLUE': [52],
 '218_GREEN': [5

#### Remove the single day columns on S2 that already exist on Triplesat
As we want to initially experiment on the enhancements through fusion of S2 and VHR data, and as we happen to already have day 266 of 2017 on S2, we will drop the 4 corresponding columns.

In [262]:
s2 = s2.drop(['266_BLUE', '266_GREEN', '266_RED', '266_NIR'], axis=1)

And then, let's re-create the `s2_colIndexes` dictionary:

In [263]:
s2_colIndexes = prepareFusionMetadata(s2)
s2_colIndexes

No duplicate columns.


{'083_BLUE': [0],
 '083_GREEN': [1],
 '083_RED': [2],
 '083_NIR': [3],
 '086_BLUE': [4],
 '086_GREEN': [5],
 '086_RED': [6],
 '086_NIR': [7],
 '116_BLUE': [8],
 '116_GREEN': [9],
 '116_RED': [10],
 '116_NIR': [11],
 '126_BLUE': [12],
 '126_GREEN': [13],
 '126_RED': [14],
 '126_NIR': [15],
 '146_BLUE': [16],
 '146_GREEN': [17],
 '146_RED': [18],
 '146_NIR': [19],
 '153_BLUE': [20],
 '153_GREEN': [21],
 '153_RED': [22],
 '153_NIR': [23],
 '156_BLUE': [24],
 '156_GREEN': [25],
 '156_RED': [26],
 '156_NIR': [27],
 '188_BLUE': [28],
 '188_GREEN': [29],
 '188_RED': [30],
 '188_NIR': [31],
 '191_BLUE': [32],
 '191_GREEN': [33],
 '191_RED': [34],
 '191_NIR': [35],
 '198_BLUE': [36],
 '198_GREEN': [37],
 '198_RED': [38],
 '198_NIR': [39],
 '201_BLUE': [40],
 '201_GREEN': [41],
 '201_RED': [42],
 '201_NIR': [43],
 '203_BLUE': [44],
 '203_GREEN': [45],
 '203_RED': [46],
 '203_NIR': [47],
 '216_BLUE': [48],
 '216_GREEN': [49],
 '216_RED': [50],
 '216_NIR': [51],
 '218_BLUE': [52],
 '218_GREEN': [5

And after this preprocessing, we have the Sentinel-2 (`s2`) and the 3SAT (`df3s`) datasets in a format that makes sense to fuse.

To create the fused dataset we will follow a procedure that is similar to what we did earlier to keep the unique VHR dataset columns.

In [270]:
# Create the new, fused dataset.
# - start by copying the index column from one of the originals. Doesn't
#   matter if it is S2 or triplesat as they share the same index
fused = pd.DataFrame(index=df3s.index)

# Create a merged list of column names
s2_cols = list(s2_colIndexes.keys())
triplesat_cols = list(triplesat_colIndexes.keys())
fused_columns = sorted(s2_cols + triplesat_cols)

# Got through the fused columns
for col in fused_columns:
    # check from which satellite (s2 or triplesat)
    if col in s2_cols:
        fused = pd.concat([fused, s2[col]], axis=1)
    elif col in triplesat_cols:
        fused = pd.concat([fused, triplesat[col]], axis=1)
    else:
        print('Warning -- Column {0} not in any of the 2 datasets. This was unexpected!'.format(col))

print("Datasets fused on the measurement level (ie. 1st level)")

Datasets fused on the measurement level (ie. 1st level)


In [271]:
fused

,083_BLUE,083_GREEN,083_NIR,083_RED,086_BLUE,086_GREEN,086_NIR,086_RED,116_BLUE,116_GREEN,...,313_NIR,313_RED,321_BLUE,321_GREEN,321_NIR,321_RED,323_BLUE,323_GREEN,323_NIR,323_RED
id,,,,,,,,,,,,,,,,,,,,,
1,641.72500,864.30000,2696.8500,845.65000,484.82500,742.30000,2546.2500,761.97500,1832.888900,2062.389000,...,1882.280000,289.315296,171.882350,290.794130,1584.000000,256.676480,171.88235,290.79413,1584.0000,256.67648
2,533.67303,807.45890,3803.0308,594.94867,375.64460,675.77264,3726.8777,494.36835,520.222200,784.628970,...,2651.885600,337.406560,241.500750,490.362550,2490.596000,273.020800,537.88983,850.35364,3424.3948,578.12665
9,493.35098,730.10583,3564.6420,488.44620,284.04095,587.36176,4151.3480,330.35324,296.216300,611.542240,...,4668.259383,347.339597,363.167245,696.928583,4201.283317,359.755983,361.50684,692.48120,4084.5393,362.86008
10,436.41833,696.79370,3662.0230,431.42120,306.54440,613.44130,3974.0230,370.11176,279.301580,537.433375,...,4570.754417,352.459903,365.103637,700.349067,4088.225283,367.846717,363.47852,697.56730,3967.5930,371.69342
114,431.49600,671.66930,4224.0293,381.00266,282.40268,539.32800,4347.9040,277.84268,362.838672,669.807998,...,4853.589400,402.208000,213.373340,453.965330,3720.453400,201.472000,257.73868,569.61600,4360.7200,253.88266
115,427.12122,665.34436,4205.0576,374.91183,279.27548,523.16530,4284.7656,272.29752,324.868445,628.654975,...,4637.289000,417.366400,236.159770,510.013760,4135.911600,227.407710,268.36365,560.20660,4134.6997,258.08540


In [265]:
fused = pd.DataFrame(index=df3s.index)
fused[]

""
id
1
2
9
10
114
115


In [267]:
fused['1'] = s2['116_GREEN']

In [268]:
fused

,1
id,
1,2062.389000
2,784.628970
9,611.542240
10,537.433375
114,669.807998
115,628.654975


In [236]:
s2['116_GREEN']

id
1      2062.389000
2       784.628970
114     669.807998
115     628.654975
Name: 116_GREEN, dtype: float64